# Behavior Cloning

## Objective: Fly balloon to the target.


0. You should **download** your ERA5 wind field data from the following link:
https://drive.google.com/file/d/1Y5ocrCH0TVf9ZDkVGaJ03ftUrtKEVxIf/view?usp=sharing
- Download the data and save it into your Google Drive.

1.Imitation learning is a paradigm where an agent learns to mimic an **expert's behavior** instead of learning purely from trial-and-error rewards (e.g., reinforcement learning).

- The simplest common approach is called "**behavior cloning**" (BC). It treats the expert's demonstrations as training data samples; train a policy using the expert's input(state) and output(action) pairs via supervised learning.

In [4]:
# Clone the repository
!git clone https://github.com/sdean-group/balloon-outreach.git
%cd balloon-outreach
!git checkout v2_learning

Cloning into 'balloon-outreach'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 927 (delta 108), reused 110 (delta 99), pack-reused 754 (from 2)
Receiving objects: 100% (927/927), 75.44 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (554/554), done.
/content/balloon-outreach/balloon-outreach
Branch 'v2_learning' set up to track remote branch 'v2_learning' from 'origin'.
Switched to a new branch 'v2_learning'


In [5]:
!pip install xarray==2025.4.0
!pip install netCDF4
!pip install opensimplex

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys
sys.path.append('/content/balloon-outreach/')

datapath = "/content/drive/MyDrive/era5_data.nc"

In [8]:
import xarray as xr
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import time

from env.balloon_env import BalloonERAEnvironment
from agent.mppi_agent import MPPIAgentWithCostFunction, MPPIAgent
from exp.learning_util import plot_agent_trajectory, plot_expert_summary
from env.util import haversine_distance

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

## Setup Environment

In [9]:
ds          = xr.open_dataset(datapath, engine="netcdf4")
start_time  = dt.datetime(year=2024, month=7, day=1, hour=0, minute=0)


#This is Ithaca
initial_lat = 42.6
initial_lon = -76.5
initial_alt = 10.0

target_lat = 77.3
target_lon = -80.5
target_alt = 12.0

time_step = 120 #120 seconds
objective = 'target'

env = BalloonERAEnvironment(ds=ds,
                            start_time=start_time,
                            initial_lat=initial_lat,
                            initial_lon=initial_lon,
                            initial_alt=initial_alt,
                            target_lat=target_lat,
                            target_lon=target_lon,
                            target_alt=target_alt,
                            objective=objective,
                            dt=time_step,
                            viz=False)


WindField initialized with noise enabled: True


In [10]:
env.wind_field.disable_noise()
initial_state = env.reset()
print(initial_state) # 21-dimensional vector

# state = (latitude, longitude, altitude, volume/max_volume, sand/max_sand,
# vertical velocity, current_time, uv vectors in 7 different pressure levels)

[ 42.6        -76.5         10.           1.           1.
   0.           0.           2.68826294  -2.50693054  11.1584198
  -7.97099304  14.01635132  -5.00876465  27.14085999   3.41699524
  34.82033386  14.09255371  38.58664856   7.6340332   18.78843994
  -4.25058899]


## Define expert policy

In [11]:
# Parameters for MPPI agent

max_steps = int(1440/(time_step/60)) #1 day

num_samples=10
acc_bounds= (-0.1, 0.1)
noise_std = 0.1
num_iterations=1

# For target
horizon=10

# For fly
# horizon = 5

In [12]:
expert = MPPIAgentWithCostFunction(target_lat=target_lat,
                                   target_lon=target_lon,
                                   target_alt=target_alt,
                                   num_samples=num_samples,
                                   acc_bounds= acc_bounds,
                                   noise_std=noise_std,
                                   num_iterations=num_iterations,
                                   horizon=horizon,
                                   visualize=False,
                                   objective=objective)

## Collect expert state-action pair

In [15]:
# Run an episode from an expert and collect its behavior

def run_expert_episode(
        env: BalloonERAEnvironment,
        agent:MPPIAgent,
        max_steps: int = 100,
        policy_name: str='expert'):
    """
    Run one episode with the given agent,
    collect state-action pair of the agent,
    and plot its trajectory in the given environment.
    """
    # Save state-action pairs from expert policy
    initial_states = []
    initial_actions = []

    state = env.reset()
    total_reward = 0

    # Store trajectory for plotting
    trajectory = [(state[0], state[1])]  # (lat, lon) pairs
    altitudes = [state[2]]  # Store altitudes

    actions = []
    velocities = []
    helium_mass = []
    sands = []
    for step in range(max_steps):
        # Get action from agent
        action = agent.select_action(state, env, step)

        # record state and expert action
        initial_states.append(state)
        initial_actions.append(action)

        # Take step
        state, reward, done, info = env.step(action)
        total_reward += reward

        actions.append(float(action[0]) if isinstance(action, np.ndarray) else float(action))
        velocities.append(env.balloon.vertical_velocity)
        helium_mass.append(env.balloon.helium_mass)
        sands.append(env.balloon.sand)

        # Store position and altitude
        trajectory.append((state[0], state[1]))
        altitudes.append(state[2])
        print(f"Step {step}: lat: {state[0]:.2f}, lon: {state[1]:.2f}, alt: {state[2]:.2f}")
        # env.render()

        if done:
            print(f"\nEpisode terminated: {info}")
            break

    # Convert to arrays
    states_np = np.array(initial_states, dtype=np.float32)
    actions_np = np.array(initial_actions, dtype=np.float32)
    print(f"Collected {len(initial_states)} state-action pairs from expert.")

    # --- Combined 2x2 Summary Plot ---
    plot_expert_summary(agent, trajectory, altitudes, actions, velocities, helium_mass, sands, 'expert', max_steps, time_step)

    return total_reward, states_np, actions_np

In [16]:
env.wind_field.disable_noise()
# env.wind_field.enable_noise(noise_seed=10) # If you want to enable noisy wind field
expert_total_reward_list = []
expert_states_list = []
expert_actions_list = []
num_iter = 1
for i in range(num_iter):
    expert_total_reward, expert_states_np, expert_actions_np = run_expert_episode(env, expert, max_steps=max_steps, policy_name=f'expert_noiseless_{i+1}')
    expert_total_reward_list.append(expert_total_reward)
    expert_states_list.append(expert_states_np)
    expert_actions_list.append(expert_actions_np)

Step 0: lat: 42.78, lon: -76.49, alt: 9.88
Step 1: lat: 42.95, lon: -76.48, alt: 9.76
Step 2: lat: 43.13, lon: -76.47, alt: 9.64
Step 3: lat: 43.31, lon: -76.46, alt: 9.52
Step 4: lat: 43.49, lon: -76.44, alt: 9.40
Step 5: lat: 43.67, lon: -76.43, alt: 9.28
Step 6: lat: 43.86, lon: -76.42, alt: 9.16
Step 7: lat: 44.04, lon: -76.40, alt: 9.04
Step 8: lat: 44.23, lon: -76.39, alt: 8.92
Step 9: lat: 44.42, lon: -76.37, alt: 8.82
Step 10: lat: 44.60, lon: -76.36, alt: 8.71
Step 11: lat: 44.79, lon: -76.34, alt: 8.60
Step 12: lat: 44.97, lon: -76.32, alt: 8.50
Step 13: lat: 45.14, lon: -76.30, alt: 8.42
Step 14: lat: 45.31, lon: -76.29, alt: 8.34
Step 15: lat: 45.47, lon: -76.27, alt: 8.28
Step 16: lat: 45.62, lon: -76.26, alt: 8.22
Step 17: lat: 45.75, lon: -76.25, alt: 8.18
Step 18: lat: 45.88, lon: -76.24, alt: 8.15
Step 19: lat: 45.99, lon: -76.23, alt: 8.13
Step 20: lat: 46.10, lon: -76.22, alt: 8.12
Step 21: lat: 46.19, lon: -76.21, alt: 8.12
Step 22: lat: 46.28, lon: -76.21, alt: 8.1

In [17]:
print(expert_total_reward_list)
expert_avg_total_reward = sum(expert_total_reward_list)/len(expert_total_reward_list)
print(f"Expert Trajectory reward in average: {expert_avg_total_reward:.2f}")

[np.float64(-8257.189702278854)]
Expert Trajectory reward in average: -8257.19


In [18]:
expert_states_np = np.concatenate(expert_states_list)
expert_actions_np = np.concatenate(expert_actions_list)

## Define our policy network

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, input_dim=21, hidden_dim=64, output_dim=1):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim*2)
        self.ln1 = nn.LayerNorm(hidden_dim*2)
        self.fc2 = nn.Linear(hidden_dim*2, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

        # Weight initialization
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
        nn.init.zeros_(self.fc2.bias)
        nn.init.xavier_uniform_(self.fc3.weight, gain=nn.init.calculate_gain('tanh'))
        nn.init.zeros_(self.fc3.bias)

    def forward(self, x):
        # x shape: (batch_size, input_dim)
        x = self.fc1(x)
        x = self.ln1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return F.tanh(x)

# Initialize policy network and optimizer
policy = PolicyNet()
optimizer = optim.Adam(policy.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

In [ ]:
# Evaluate a policy on the environment
def evaluate_policy(env: BalloonERAEnvironment,
                    policy: nn.Module,
                    max_steps: int,
                    policy_name: str,
                    expert_avg_total_reward: float):
    policy.eval()
    if policy.training:
        print("→ policy is in training mode")
    else:
        print("→ policy is in evaluation mode")

    state = env.reset()
    total_reward = 0.0

    # Store trajectory for plotting
    trajectory = [(state[0], state[1])]  # (lat, lon) pairs
    altitudes = [state[2]]  # Store altitudes

    actions = []
    velocities = []
    helium_mass = []
    sands = []

    for step in range(max_steps):
        state_tensor = torch.from_numpy(state).float().unsqueeze(0)      # shape (1, 21)
        action_pred = policy(state_tensor).item()                        # continuous action
        state, reward, done, info = env.step(action_pred)
        total_reward += reward

        actions.append(float(action_pred) if isinstance(action_pred, np.ndarray) else float(action_pred))
        velocities.append(env.balloon.vertical_velocity)
        helium_mass.append(env.balloon.helium_mass)
        sands.append(env.balloon.sand)

        # Store position and altitude
        trajectory.append((state[0], state[1]))
        altitudes.append(state[2])

        if done:
            print(f"\nEpisode terminated: {info}")
            break
    print(f"Total reward obtained from current policy: {total_reward:.2f}")
    print(f"Expert policy reward - Current policy reward: {expert_avg_total_reward-total_reward:.2f}")

    # Plot final trajectory
    plot_agent_trajectory(trajectory=trajectory,
                          altitudes=altitudes,
                          actions=actions,
                          velocities=velocities,
                          helium_mass=helium_mass,
                          sands=sands,
                          policy_name=policy_name,
                          max_steps=max_steps,
                          dt=env.dt,
                          objective='target',
                          target_lon=env.target_lon,
                          target_lat=env.target_lat,
                          target_alt=env.target_alt
                          )

    return total_reward

In [ ]:
env.wind_field.disable_noise()
# env.wind_field.enable_noise(noise_seed=10) # If you want to enable noisy wind field
evaluate_policy(env, policy, max_steps=max_steps, policy_name='random_policy_noiseless', expert_avg_total_reward=expert_avg_total_reward)

## Behavior Cloning (Supervised Learning)

In [ ]:
# === Initial Dataset & DataLoader Construction ===

# Convert to tensors
states_tensor = torch.from_numpy(expert_states_np).float()               # shape (N, 21)
actions_tensor = torch.from_numpy(expert_actions_np).float()             # shape (N, 1)

# Construct TensorDataset and DataLoader
dataset = TensorDataset(states_tensor, actions_tensor)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# === Training Function Using Loader ===
def train_one_epoch(loader, policy, optimizer, loss_fn):
    policy.train()
    if policy.training:
        print("→ policy is in training mode")
    else:
        print("→ policy is in evaluation mode")

    total_loss = 0.0
    for x_batch, y_batch in loader:
        optimizer.zero_grad()
        preds = policy(x_batch)
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x_batch.size(0)
    avg_loss = total_loss / len(loader.dataset)
    return avg_loss

In [ ]:
losses = []
for epoch in range(10):
    avg_loss = train_one_epoch(loader, policy, optimizer, loss_fn)
    losses.append(avg_loss)
    print(f"Epoch {epoch+1}/10 — Avg Loss: {avg_loss:.6f} \n")

# plot the loss curve from behavior cloning
plt.figure(figsize=(6,4))
plt.plot(range(1, len(losses)+1), losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Average Training Loss')
plt.title('Behavior Cloning Loss Curve')
plt.grid(True)
plt.show()

In [ ]:
env.wind_field.disable_noise()
# env.wind_field.enable_noise(noise_seed=10) # If you want to enable noisy wind field
evaluate_policy(env, policy, max_steps=max_steps, policy_name='BC_trained_policy_noiseless_wf', expert_avg_total_reward=expert_avg_total_reward)